In [11]:
import os
import cohere
from langchain_openai import OpenAIEmbeddings
from langchain_postgres.vectorstores import PGVector
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
os.environ["OPENAI_API_KEY"] =os.getenv("OPENAI_API_KEY")
cohere_api_key = os.getenv("COHERE_API_KEY")
POSTGRES_URL = os.getenv("POSTGRES_URL")

co = cohere.Client(cohere_api_key)

In [12]:
def get_vectorstore_from_postgres(collection_name):
    openai_ef = OpenAIEmbeddings()
    vectorstore = PGVector(
        embeddings=openai_ef,
        collection_name=collection_name,
        connection=POSTGRES_URL,
        use_jsonb=True,
    ) 
    return vectorstore  

In [4]:
vectorstore=get_vectorstore_from_postgres("c1")

In [13]:
def get_context_from_vectorstore(user_query,vectorstore):
    relevant_docs = vectorstore.similarity_search(user_query,k=20)
    docs = [x.page_content for x in relevant_docs]
    return docs

In [14]:
query="what is the value of Total Community Services in adopted 2023 2024"
docs=get_context_from_vectorstore(query,vectorstore)

In [26]:
def cohere_re_ranker(query, docs):
    rerank_docs = co.rerank(query=query, documents=docs, top_n=4, model='rerank-english-v3.0')
    return rerank_docs

In [27]:
rerank_docs=cohere_re_ranker(query,docs)

In [30]:
rerank_docs.results

[RerankResponseResultsItem(document=None, index=2, relevance_score=0.97389406),
 RerankResponseResultsItem(document=None, index=5, relevance_score=0.8392076),
 RerankResponseResultsItem(document=None, index=6, relevance_score=0.8071568),
 RerankResponseResultsItem(document=None, index=9, relevance_score=0.67768735)]